In [1]:
import psycopg2
import pandas as pd

host = 'localhost'
dbname = 'nyc-taxi-data'
user = 'cg'

try:
    conn_str = f"host={host} dbname={dbname} user={user}"
    conn = psycopg2.connect(conn_str)
    
except Exception as e:
    print(e)

In [2]:
sql = """SELECT A.id, 
                EXTRACT(YEAR FROM A.pickup_datetime) AS year, 
                EXTRACT(MONTH FROM A.pickup_datetime) AS month, 
                EXTRACT(DOW FROM A.pickup_datetime) AS dow, 
                A.pickup_location_id, 
                CASE 
                    WHEN A.dropoff_location_id IS NULL THEN 1
                    ELSE 0
                END AS no_do_location,
                UPPER(B.dba_category) AS type
            FROM fhv_trips_samp_pt1pct A 
            LEFT JOIN fhv_bases B 
            ON A.dispatching_base_num = B.base_number;"""
df = pd.read_sql_query(sql, conn)
df.head()

,id,year,month,dow,pickup_location_id,no_do_location,type
0,86216845,2016.0,2.0,5.0,7.0,1,UBER
1,94385731,2016.0,3.0,3.0,262.0,1,VIA
2,122667468,2016.0,6.0,0.0,36.0,1,UBER
3,134508720,2016.0,7.0,5.0,225.0,1,UBER
4,160305142,2016.0,9.0,4.0,236.0,1,UBER


In [3]:
pd.crosstab([df.year, df.type], df.no_do_location)

no_do_location       0      1
year   type                  
2014.0 UBER          0   4566
2015.0 LYFT          0   2650
       OTHER         0  22195
       UBER          0  36966
       VIA           0   1809
2016.0 JUNO          0   4994
       LYFT          0  11338
       OTHER         0  40132
       UBER          0  70317
       VIA           0   6304
2017.0 JUNO          0  12442
       LYFT      17988   8526
       OTHER     14467  18704
       UBER      69435  40655
       VIA        6549   3702
2018.0 JUNO          0  13669
       LYFT      44741      0
       OTHER     22000   5875
       UBER     163285    109
       VIA       11369      0

In [4]:
pd.crosstab([df.year, df.type], df.no_do_location).apply(lambda r: r/r.sum(), axis=1)

no_do_location         0         1
year   type                       
2014.0 UBER     0.000000  1.000000
2015.0 LYFT     0.000000  1.000000
       OTHER    0.000000  1.000000
       UBER     0.000000  1.000000
       VIA      0.000000  1.000000
2016.0 JUNO     0.000000  1.000000
       LYFT     0.000000  1.000000
       OTHER    0.000000  1.000000
       UBER     0.000000  1.000000
       VIA      0.000000  1.000000
2017.0 JUNO     0.000000  1.000000
       LYFT     0.678434  0.321566
       OTHER    0.436134  0.563866
       UBER     0.630711  0.369289
       VIA      0.638865  0.361135
2018.0 JUNO     0.000000  1.000000
       LYFT     1.000000  0.000000
       OTHER    0.789238  0.210762
       UBER     0.999333  0.000667
       VIA      1.000000  0.000000

In [5]:
conn.close()